In [1]:
import pandas as pd
import numpy as np
import ipywidgets as ipw
import plotly.graph_objects as go

import tardis
from tardis.visualization import CustomAbundanceWidget
from tardis.util.base import (
    atomic_number2element_symbol,
    species_tuple_to_string,
)

/Users/yuyizheng/opt/anaconda3/envs/tardis/lib/python3.7/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)


In [2]:
# sim = tardis.run_tardis('my_yml.yml')

### Load from Simulation object

In [3]:
# CustomAbundanceWidget.from_simulation(sim)

### If config file contains 'csvy_model' module, 'from_yml()' fails

In [4]:
# CustomAbundanceWidget.from_yml('my_yml.yml')

### Call 'from_csvy()' then

In [5]:
# my_cawidget = CustomAbundanceWidget.from_csvy('my_csvy.csvy')

### How abundance data is stored in this class

In [6]:
# my_cawidget.abundance

### Load from YAML file

In [7]:
cawidget = CustomAbundanceWidget.from_yml('tardis_example.yml')

In [8]:
cawidget.abundance

,,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
atomic_number,mass_number,,,,,,,,,,,,,,,,,,,,
8,,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19
12,,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
14,,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52
16,,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19
18,,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04
20,,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03


### Codes to generate GUI

In [9]:
abundance = cawidget.abundance.copy()
velocity = cawidget.velocity.copy()
no_of_elements = abundance.shape[0]
no_of_shells = abundance.shape[1]

def get_symbols(abundance):
    str_symbols = np.array(abundance.index.get_level_values(0).map(atomic_number2element_symbol))
    str_mass = np.array(abundance.index.get_level_values(1), dtype='str')
    return(np.add(str_symbols, str_mass))

elements = get_symbols(abundance)

In [10]:
from IPython.display import display

fig = go.FigureWidget()

l = list(range(1, no_of_shells+1))
dropdown = ipw.Dropdown(options=l, 
                        description='Shell No. ', 
                        layout=ipw.Layout(width='160px')
                       )

# shell_no = dropdown.value

btn_previous = ipw.Button(icon="chevron-left", 
                          disabled=True, 
                          layout=ipw.Layout(width='30px', height='30px')
                         )
btn_next = ipw.Button(icon="chevron-right", 
                      layout=ipw.Layout(width='30px', height='30px')
                     )

head = ipw.HBox([dropdown, btn_previous, btn_next])

locks = [ipw.Checkbox(indent=False, 
                      layout=ipw.Layout(width='50px')
                     ) for element in elements]

items = [ipw.BoundedFloatText(min=0, 
                              max=1, 
                              step=0.01, 
                              description=element) 
         for element in elements]

btn_normalize = ipw.Button(description='Normalize',
                           icon='cog', 
                           layout=ipw.Layout(margin='0 0 0 100px')
                          )
abundance_edit_container = ipw.HBox([ipw.VBox(items), ipw.VBox(locks)])
# abundance_edit_container = ipw.HBox([ipw.VBox(items), ipw.VBox(locks, layout=ipw.Layout(width='20%'))], layout=ipw.Layout(width='50%'))

In [11]:
# generate abundance vs velocity plot using Plotly
plot_output = ipw.Output()
def generate_abundance_plot():
    plot_output.clear_output()
    with plot_output:
        title = "Abundance vs Velocity"
        data = abundance.T

        for i in range(no_of_elements):
            fig.add_trace(
                go.Scatter(
                    x=velocity[1:] / 10e5, # convert to km/s
                    y=data.iloc[:,i],
                    mode="lines+markers",
                    name=elements[i],
                )
            )

        fig.update_layout(
            xaxis=dict(
                title="Velocity of outer boundary (km/s)",
                exponentformat="e",
            ),
            yaxis=dict(title="Fractional Abundance", 
                       exponentformat="e"),
            yaxis_range = [0, 1],
            height=500,
            title=title,
        )

        display(fig)

In [12]:
locked_list = np.array([False] * no_of_elements)

def lock_checkbox_handler(obj):
    abundance_index = obj.owner.name
    shell_no = dropdown.value
    
    if obj.new == True:
        locked_list[abundance_index] = True
        
        # Ensure the sum of locked elements less than 1
        locked_sum = abundance.loc[locked_list, shell_no-1].sum()
        if locked_sum > 1:
            items[abundance_index].value = 1 - (abundance.loc[locked_list, shell_no-1].sum() - items[abundance_index].value)
    
    else:
        locked_list[abundance_index] = False

In [13]:
def update_abundance(obj):
    shell_no = dropdown.value
    element_no = obj.owner.name # start from 0

    # Ensure the sum of locked elements less than 1
    if locked_list[element_no] == True: # if the element is locked
        locked_sum = abundance.loc[locked_list, shell_no-1].sum() - obj.old + obj.new
        if locked_sum > 1:
            obj.new = 1 - (abundance.loc[locked_list, shell_no-1].sum() - obj.old)
            obj.owner.value = obj.new
    
    abundance.iloc[element_no, shell_no-1] = obj.new
    
    # Update plot
    fig.data[element_no].y = abundance.iloc[element_no]
    
def read_abundance(shell_no):
    for i, item in enumerate(items):
        item.value = abundance.iloc[i,shell_no-1]

# def display_abundance_df():
#     abundance_view = abundance.copy()
#     abundance_view.index = elements
#     abundance_view.columns=range(1, no_of_shells+1)
#     abundance_view.columns.name = 'Shell No.'
#     display(abundance_view)

In [14]:
# out = ipw.Output()

# Change the shell
def dropdown_eventhandler(obj):   
    locked_list = np.array([False] * no_of_elements)
    for lock in locks:
        lock.value = False 
    shell_no_new = obj.new
    read_abundance(shell_no_new)
    
    # Control 'previous' and 'next' buttons.
    if obj.new == 1:
        btn_previous.disabled = True
    else:
        btn_previous.disabled = False
    
    if obj.new == no_of_shells:
        btn_next.disabled = True
    else:
        btn_next.disabled = False

dropdown.observe(dropdown_eventhandler, names='value')

for i, item in enumerate(items):
    item.observe(update_abundance, names='value')
    item.name = i
    locks[i].observe(lock_checkbox_handler, names='value')
    locks[i].name = i

In [15]:
def to_previous_shell(obj):
    dropdown.value -= 1
    
def to_next_shell(obj):
    dropdown.value += 1

def normalize(obj):
    shell_no = dropdown.value
    locked_sum = abundance.loc[locked_list, shell_no-1].sum()
    
    # if abundances are all zero
    if abundance.loc[~locked_list, shell_no-1].sum() == 0:
        return
    
    abundance.loc[~locked_list, shell_no-1] = (1 - locked_sum) * abundance.loc[~locked_list, shell_no-1] / abundance.loc[~locked_list, shell_no-1].sum()
    #abundance.iloc[:, shell_no-1] = abundance.iloc[:, shell_no-1] / abundance.iloc[:, shell_no-1].sum()
    
    read_abundance(shell_no)
    

btn_normalize.on_click(normalize)
btn_previous.on_click(to_previous_shell)
btn_next.on_click(to_next_shell)

In [16]:
# def reset_widgets():
#     pass

In [17]:
help_note = ipw.HTML(value="<p style=\"text-indent: 40px\"><b>Help:</b> Click the checkbox to lock the abundance\n\n you don't want to normalize.</p>",
                        indent=True
                        )

### Additional Feature 1: Add new elements

In [18]:
import asyncio

class Timer:
    """
    Cited from https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html
    """
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback

    async def _job(self):
        await asyncio.sleep(self._timeout)
        self._callback()

    def start(self):
        self._task = asyncio.ensure_future(self._job())

    def cancel(self):
        self._task.cancel()

def debounce(wait):
    """ 
    Decorator that will postpone a function's
    execution until after `wait` seconds
    have elapsed since the last time it was invoked. 
    Cired from https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html
    """
    def decorator(fn):
        timer = None
        def debounced(*args, **kwargs):
            nonlocal timer
            def call_it():
                fn(*args, **kwargs)
            if timer is not None:
                timer.cancel()
            timer = Timer(wait, call_it)
            timer.start()
        return debounced
    return decorator

In [19]:
from pyne import nucname

In [20]:
invalid = ipw.Valid(value=False,
                    description='',
                    style={'description_width': 'initial'},
                    layout=ipw.Layout(visibility='hidden',
                                      width='250px')
                    )
symbol_text = ipw.Text(description='Element: ',
                       placeholder='symbol',
                       layout=ipw.Layout(width='200px')
                      )
symbol_box = ipw.HBox([symbol_text, invalid])

mass_text = ipw.BoundedFloatText(
                                value=0,
                                min=0,
                                max=1.0,
                                step=0.1,
                                description='Abundance: ',
                                layout=ipw.Layout(width='200px')
                            )

btn_add_element = ipw.Button(icon='plus-square', 
                             description=' New Element',
                             disabled=True,
                             layout=ipw.Layout(margin='0 0 0 50px')
                            )

In [21]:
@debounce(0.5)
def symbol_validation(obj):
    element_symbol_string = obj.new.capitalize()
    
    if element_symbol_string in elements:        
        invalid.description = 'Already exists!'
        invalid.layout.visibility = 'visible'
        btn_add_element.disabled = True
        return
    
    try:
        if nucname.iselement(element_symbol_string) or nucname.isnuclide(element_symbol_string):
            invalid.layout.visibility = 'hidden'
            btn_add_element.disabled = False
        elif element_symbol_string == "":
            invalid.layout.visibility = 'hidden' 
            btn_add_element.disabled = True
   
        else:
            invalid.description = ''
            invalid.layout.visibility = 'visible'
            btn_add_element.disabled = True

    except RuntimeError:
        invalid.description = ''
        invalid.layout.visibility = 'visible'
        btn_add_element.disabled = True
    
symbol_text.observe(symbol_validation, names='value')

In [22]:
def add_element(obj):
    global no_of_elements, no_of_shells, locked_list, elements, abundance_edit_container, main_box
    
    element_symbol_string = symbol_text.value.capitalize()
    fractional_abundance = mass_text.value
    
    if element_symbol_string in nucname.name_zz:
        z = nucname.name_zz[element_symbol_string]
        abundance.loc[(z, ''), :] = fractional_abundance
    else:
        mass_no = nucname.anum(element_symbol_string)
        z = nucname.znum(element_symbol_string)
        abundance.loc[(z, mass_no), :] = fractional_abundance
    
    abundance.sort_index(inplace=True)
    no_of_elements += 1
    no_of_shells += 1
    
    # Add new BoundedFloatText control and Checkbox control.
    new_item = ipw.BoundedFloatText(min=0, max=1, step=0.01)
    new_lock = ipw.Checkbox(indent=False, layout=ipw.Layout(width='50px'))
    new_item.name = no_of_elements - 1
    new_lock.name = no_of_elements - 1
    items.append(new_item)
    locks.append(new_lock)
    
    locked_list = np.append(locked_list, False)
    
    # Keep the order of description same with atomic number
    elements = get_symbols(abundance)
    for i in range(no_of_elements):
        items[i].description = elements[i]

    abundance_edit_container.children = [ipw.VBox(items), ipw.VBox(locks)]
    
    # Add new trace to plot.
    fig.add_scatter(x=velocity[1:] / 10e5, # convert to km/s
                    y=[fractional_abundance]*no_of_shells,
                    mode="lines+markers",
                    name=element_symbol_string,
                   )
    
    read_abundance(dropdown.value)
        
    new_item.observe(update_abundance, names='value')
    new_lock.observe(lock_checkbox_handler, names='value')
    
    # Recover the text boxes.
    symbol_text.value = ''
    mass_text.value = 0
    
btn_add_element.on_click(add_element)

In [23]:
add_element_box = ipw.VBox([symbol_box, mass_text, btn_add_element], layout=ipw.Layout(margin='0 0 0 100px'))
main_box = ipw.HBox([abundance_edit_container, add_element_box])
# main_output = ipw.Output()
# with main_output:
#     display(main_box)

In [24]:
# Display the widget.
generate_abundance_plot()
display(plot_output)
display(head, help_note)
read_abundance(shell_no=1)
display(main_box, btn_normalize)

Output()

HTML(value='<p style="text-indent: 40px"><b>Help:</b> Click the checkbox to lock the abundance\n\n you don\'t …

Button(description='Normalize', icon='cog', layout=Layout(margin='0 0 0 100px'), style=ButtonStyle())